Branje podatkov iz datotek in uvoz knjižnic:

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np


In [ ]:
list_of_cvs = ["./podatki/kd{}.csv".format(i) for i in range(2010,2020)]
frames = [pd.read_csv(csv,sep=';', encoding="ANSI") for csv in list_of_cvs]
df = pd.concat(frames)


Količina vpletenih v posamezna kazniva dejanja:

In [ ]:
#Stevilo vpletenih v dejanje
size = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD'])
vpletenih = size.size()
print(vpletenih.agg(["min","max","mean"]))
plt.hist(vpletenih, bins=500)
plt.show()


Kazniva dejanja po mesecih:

In [ ]:
#group by mesec, count unique id
kd_mes = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD']).first()
mes_counter = defaultdict(int)

for v in kd_mes.values:
    print(v)
    #print((v.astype(str)).split(".")[0])
    #mes_counter[int(v.astype(str).split(".")[0].strip())] +=1

""" list_mes_counter = sorted(mes_counter.items())
mesec, counter = zip(*list_mes_counter)
plt.plot(mesec,counter)
plt.show() """